In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup 
import pandas as pd
import csv

# Open web automaticly
driver=webdriver.Edge()
url='https://www.thegioididong.com' #Lấy link để mở web tự động
driver.get(url)
dt=driver.find_element_by_xpath('/html/body/header/div[2]/div/ul/li[1]/a/span') 
dt.click()


#Scrape data of products from soucre web 
def GetUrl():
    page_s=BeautifulSoup(driver.page_source) #dùng bs4 tải toàn bộ HTML document lên chương trình
    tel_infors=page_s.find_all('a',class_='main-contain') # tìm đúng element trong source để lấy chính xác id, ở đây sử dụng class và  task a
   
    all_prod=[]
    for tel_infor in tel_infors:
        tel_ID=tel_infor.get('href') # lấy link đuôi trong thành phần có tên là href
        tel_If_url='https://www.thegioididong.com'+tel_ID
        if tel_If_url not in all_prod:  #để tránh lặp lại gá trị vì trong web có các thành phần nằm chồng lên nhau và giống nhau
            all_prod.append(tel_If_url)
    set(all_prod)
    return all_prod
#print(GetUrl(),'\n==============================================================================')



C:\Users\Admin\AppData\Local\Temp\ipykernel_10144\2473477986.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  dt=driver.find_element_by_xpath('/html/body/header/div[2]/div/ul/li[1]/a/span')


Lặp từng vòng page để lấy dữ liệu

In [5]:
#numPg=int(input('Input the page number do you want to scrape'))
numPg=5
    
Url_all=[]
  
for i in range(numPg):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)') # thực hiện kéo xuống xuối cuối trang 
    next_btn=driver.find_element_by_xpath('//*[@id="categoryPage"]/div[3]/div[2]/a')
    next_btn.click()
driver.execute_script('window.scrollTo(0,document.body.scrollHeight)') # thực hiện kéo xuống xuối cuối trang 
Url_each_page=GetUrl() # Lấy thông tin tất cả từng trang
Url_all=Url_all+Url_each_page  
print(Url_all)
 




C:\Users\Admin\AppData\Local\Temp\ipykernel_10144\363947574.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  next_btn=driver.find_element_by_xpath('//*[@id="categoryPage"]/div[3]/div[2]/a')


['https://www.thegioididong.com/dtdd/samsung-galaxy-s22-ultra?src=osp', 'https://www.thegioididong.com/dtdd/iphone-11?src=osp', 'https://www.thegioididong.com/dtdd/oppo-reno7-z?src=osp', 'https://www.thegioididong.com/dtdd/realme-c35?src=osp', 'https://www.thegioididong.com/dtdd/xiaomi-11t?src=osp', 'https://www.thegioididong.com/dtdd/samsung-galaxy-a03?src=osp', 'https://www.thegioididong.com/dtdd/xiaomi-redmi-note-11-pro-4g?src=osp', 'https://www.thegioididong.com/dtdd/xiaomi-redmi-note-11s-5g?src=osp', 'https://www.thegioididong.com/dtdd/iphone-13-pro-max?src=osp', 'https://www.thegioididong.com/dtdd/samsung-galaxy-a52s-5g?src=osp', 'https://www.thegioididong.com/dtdd/vivo-y15s-2021?src=osp', 'https://www.thegioididong.com/dtdd/samsung-galaxy-z-fold-3?src=osp', 'https://www.thegioididong.com/dtdd/iphone-12-pro-max-512gb?src=osp', 'https://www.thegioididong.com/dtdd/oppo-find-x5-pro?src=osp', 'https://www.thegioididong.com/dtdd/iphone-13-pro?src=osp', 'https://www.thegioididong.com/d

Write into file .csv

In [ ]:

with open('Du_lieu_dt_7.csv','w',newline='',encoding='utf-8')as file_output:
    header=['Name','Brand','Display','HDH','Camera sau','Camera trước','Chip','Ram','Rom','Sim','Battery','Charge','Gia','Danh gia','So Luong','Link','Tên cửa hàng']
    writer =csv.DictWriter(file_output,fieldnames=header)
    writer.writeheader()
    for url in Url_all:
        driver.get(url)
        page_source=BeautifulSoup(driver.page_source,"html.parser")
        display=[]
        div1=page_source.find('div',class_='parameter')
        try :
            name=page_source.find('h1').get_text()
            tmp_brand=page_source.find('h1').get_text()
            brand=tmp_brand.split(' ')[2]
            
        except AttributeError:
            name='null'
            continue
        for a in div1.find('div',class_='liright'):
            if a.get_text()!='\n':
                display.append(a.get_text())
        display2= ",".join(display)#Thông tin màn hình
        path_li=div1.find_all('li')
        #print(type(hdh_path))
        #vì trong các thành phần được trích xuất từ bs4 có dạng mảng nên ta có thể trích xuất các phần tử
        # ở đây ta trích xuất phần tử từ trên xuống theo các phần tử có được trích xuất từ web 
        hdh=path_li[1].find('span').get_text() #Lấy thông tin hệ điều hành
        camera_sau=path_li[2].find('span').get_text() #Lấy thông tin camera sau
        camera_truoc=path_li[3].find('span').get_text() #Lấy thông tin camera trước
        try :
            chip=path_li[4].find('span').get_text() #Lấy thông tin chip
        except AttributeError:
            continue
        ram=path_li[5].find('span').get_text() #Lấy thông tin ram
        rom=path_li[6].find('span').get_text() #Lấy thông tin rom
        try:
            sim=path_li[7].find('span').get_text() #Lấy thông tin sim
        except AttributeError:
            continue
        except IndexError:
            continue
        try:
            gia=page_source.find('p',class_='box-price-present').get_text() #Lấy thông tin giá
        except AttributeError:
            continue
        
        battery=''
        charge=''
        nameshop="Thế giới di động"
        link=url
        i=0
        for tmp in  path_li[8].find('div'):
            if i==0 and tmp.get_text()!='\n':
                battery=battery+tmp.get_text()
                i=i+1
            else:
                charge=charge+tmp.get_text().strip()
        try :
            danhgia=page_source.find('p',class_='point').get_text() #Lấy thông tin đánh giá
            tmp=page_source.find('p',class_="detail-rate-total").get_text() #Lấy thông tin số lượng
            tmp=int(tmp.split(' ')[0])
            sluong=tmp
            
            print(name,display2,hdh,camera_sau,camera_truoc,chip,ram,rom,sim,battery,charge,gia,danhgia,sluong)
        except AttributeError:
                danhgia='NULL'
                sluong=0
                writer.writerow({ header[0]:name,header[1]:brand,header[2]:display2,header[3]:hdh,header[4]:camera_sau,header[5]:camera_truoc,header[6]:chip,header[7]:ram,header[8]:rom,header[9]:sim,header[10]:battery,header[11]:charge,header[12]:gia,header[13]:danhgia,header[14]:sluong,header[15]:link,header[16]:nameshop})
                print(name,display2,hdh,camera_sau,camera_truoc,chip,ram,rom,sim,battery,charge,gia,danhgia,sluong)

                continue
        print(name,display2,brand,hdh,camera_sau,camera_truoc,chip,ram,rom,sim,battery,charge,gia,danhgia,sluong)
        writer.writerow({ header[0]:name,header[1]:brand,header[2]:display2,header[3]:hdh,header[4]:camera_sau,header[5]:camera_truoc,header[6]:chip,header[7]:ram,header[8]:rom,header[9]:sim,header[10]:battery,header[11]:charge,header[12]:gia,header[13]:danhgia,header[14]:sluong,header[15]:link,header[16]:nameshop})
           
    

In [2]:
import pandas as pd
pd.read_csv('dt3.csv')

,Name,Brand,Display,HDH,Camera sau,Camera trước,Chip,Ram,Rom,Sim,Battery,Gia,Danh gia,So Luong,Link,Tên cửa hàng
0,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,Samsung,"Dynamic AMOLED 2X,6.8"",Quad HD+ (2K+)",Android 12,"Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP",40 MP,Snapdragon 8 Gen 1 8 nhân,8 GB,128 GB,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh,30990000,3.0,18,https://www.thegioididong.com/dtdd/samsung-gal...,Thế giới đi động
1,Điện thoại iPhone 11 64GB,iPhone,"IPS LCD,6.1"",Liquid Retina",iOS 15,2 camera 12 MP,12 MP,Apple A13 Bionic,4 GB,64 GB,1 Nano SIM & 1 eSIM,3110 mAh,14990000,4.0,817,https://www.thegioididong.com/dtdd/iphone-11?s...,Thế giới đi động
2,Điện thoại OPPO Reno7 Z 5G,OPPO,"AMOLED,6.43"",Full HD+",Android 11,"Chính 64 MP & Phụ 2 MP, 2 MP",16 MP,Snapdragon 695 5G 8 nhân,8 GB,128 GB,2 Nano SIM (SIM 2 chung khe thẻ nhớ),4500 mAh,10490000,3.8,81,https://www.thegioididong.com/dtdd/oppo-reno7-...,Thế giới đi động
3,Điện thoại Realme C35 64GB,Realme,"IPS LCD,6.6"",Full HD+",Android 11,"Chính 50 MP & Phụ 2 MP, 0.3 MP",8 MP,Unisoc T616 8 nhân,4 GB,64 GB,2 Nano SIM,5000 mAh,3990000,3.5,94,https://www.thegioididong.com/dtdd/realme-c35?...,Thế giới đi động
4,Điện thoại Xiaomi 11T 5G 128GB,Xiaomi,"AMOLED,6.67"",Full HD+",Android 11,"Chính 108 MP & Phụ 8 MP, 5 MP",16 MP,MediaTek Dimensity 1200,8 GB,128 GB,2 Nano SIM,5000 mAh,10990000,3.8,310,https://www.thegioididong.com/dtdd/xiaomi-11t?...,Thế giới đi động
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Oppo A16,NaN,"Android 11, ColorOS 11.1","Android 11, ColorOS 11.1","Camera góc rộng: 13 MP, f/2.2 Camera macro: 2...","8 MP, f/2.0",MediaTek Helio G35,3 GB,32 GB,2 SIM (Nano-SIM),Li-Po 5000 mAh,3490000,5.0,0,https://cellphones.com.vn/oppo-a16.html,Cellphones
215,Xiaomi POCO X3 NFC 128GB,Xiaomi,"Android 10, MIUI 12","Android 10, MIUI 12","Camera chính: 64 MP, f/1.9, (wide), 1/1.73"", 0...","20 MP, f/2.2, (wide), 1/3.4"", 0.8µm",Qualcomm SM7150-AC Snapdragon 732G (8 nm),6 GB,128 GB,2 SIM (Nano-SIM),Li-Po 5160 mAh Sạc nhanh 33W,5700000,4.5,21,https://cellphones.com.vn/xiaomi-poco-x3.html,Cellphones
216,OPPO A15,OPPO,Android 10,Android 10,"13 MP, f/2.2, (wide), 1/3.1"", 1.12µm, PDAF 2 M...","8 MP, (wide)",Mediatek MT6765 Helio P35 (12nm),3 GB,32 GB,2 SIM (Nano-SIM),Li-Po 4230 mAh,3290000,4.0,3,https://cellphones.com.vn/oppo-a15.html,Cellphones
217,OPPO Reno5,OPPO,"ColorOS 11.1, Nền tảng Android 11","ColorOS 11.1, Nền tảng Android 11",64 MP + 8 MP + 2 MP + 2 MP,"44 MP, F/2.4, Cảm biến thông minh AI","Qualcomm Snapdragon 720G, tối đa 2.3GHz",8 GB,128 GB,2 SIM (Nano-SIM),"4310mAh (Typ), Sạc siêu nhanh",7790000,4.8,26,https://cellphones.com.vn/oppo-reno-5.html,Cellphones


In [ ]:

df=pd.read_csv('Du_lieu_dt_6.csv')
df['Tên cửa hàng']='Thế giới đi động'
df['Gia']=df['Gia'].replace('[*₫Giá dự kiến: .]','',regex=True)
df['Gia']=pd.to_numeric(df['Gia']).astype(int)
df=df.dropna(axis=0,how='any')
df.to_csv('Du_lieu_dt_6.csv',index=False)